In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 4096
n_epochs = 10

In [4]:
df = pd.read_csv('../options-df-sigma.csv')
df = df.dropna(axis=0)
df = df.drop(columns=['date', 'exdate', 'impl_volatility', 'volume', 'open_interest'])
df.strike_price = df.strike_price / 1000
call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [5]:
call_df.head()

,strike_price,best_bid,best_offer,date_ndiff,treasury_rate,closing_price,sigma_20
0,600.0,28.000,29.000,47,5.17,624.22,0.007761
10,475.0,152.875,153.875,145,5.12,624.22,0.007761
13,600.0,52.375,53.375,327,5.05,624.22,0.007761
17,610.0,20.000,20.750,47,5.17,624.22,0.007761
18,675.0,34.000,35.000,691,5.10,624.22,0.007761


In [6]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                        (call_df.best_bid + call_df.best_offer) / 2,
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                    (put_df.best_bid + put_df.best_offer) / 2,
                                                                    test_size=0.01, random_state=42)

In [7]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

model.add(Dense(1, activation='relu'))

model.compile(loss='mse', optimizer=Adam())

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               2400      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
batch_normalization_1 (Batch (None, 400)               1600      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               160400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 400)               1600      
__________

In [9]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)


Train on 5001241 samples, validate on 50518 samples


Epoch 1/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 33852.9942 - val_loss: 2218.8129
Epoch 2/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 134.5385 - val_loss: 3024.7805
Epoch 3/10
5001241/5001241 [==============================] - 28s 6us/step - loss: 116.8041 - val_loss: 159.8780
Epoch 4/10
5001241/5001241 [==============================] - 28s 6us/step - loss: 110.6229 - val_loss: 1200.5006
Epoch 5/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 107.3717 - val_loss: 94.1658
Epoch 6/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 109.0304 - val_loss: 1265.1090
Epoch 7/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 100.7825 - val_loss: 6988.1798
Epoch 8/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 104.4686 - val_loss: 278.3381
Epoch 9/10
5001241/5001241 [=======

In [10]:
model.save('mlp1-call10.h5')

In [11]:
model.compile(loss='mse', optimizer=Adam(lr=1e-4))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=4096, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 88.4756 - val_loss: 251.5875
Epoch 2/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 82.5439 - val_loss: 45.6215
Epoch 3/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 78.1060 - val_loss: 42.0497
Epoch 4/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 78.9330 - val_loss: 60.2502
Epoch 5/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 76.1271 - val_loss: 37.8237
Epoch 6/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 71.9409 - val_loss: 42.0201
Epoch 7/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 71.7083 - val_loss: 167.5035
Epoch 8/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 73.6494 - val_loss: 640.7643
Epoch 9/10
5001241/5001241 [=============================

In [12]:
model.save('mlp1-call20.h5')

In [13]:
model.compile(loss='mse', optimizer=Adam(lr=1e-5))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=4096, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 65.1953 - val_loss: 25.9072
Epoch 2/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 64.5409 - val_loss: 23.7328
Epoch 3/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 64.2214 - val_loss: 30.2003
Epoch 4/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 64.7132 - val_loss: 35.9980
Epoch 5/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 66.6474 - val_loss: 25.6222
Epoch 6/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 64.4956 - val_loss: 22.3697
Epoch 7/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 63.8673 - val_loss: 29.9699
Epoch 8/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 63.1887 - val_loss: 25.7095
Epoch 9/10
5001241/5001241 [==============================] 

In [14]:
model.save('mlp1-call30.h5')

In [15]:
model.compile(loss='mse', optimizer=Adam(lr=1e-6))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=4096, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 62.5827 - val_loss: 23.8257
Epoch 2/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 60.4313 - val_loss: 24.4241
Epoch 3/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 63.2473 - val_loss: 21.9690
Epoch 4/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 64.2664 - val_loss: 21.8891
Epoch 5/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 62.0405 - val_loss: 23.2404
Epoch 6/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 62.2570 - val_loss: 23.1145
Epoch 7/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 62.4362 - val_loss: 22.0084
Epoch 8/10
5001241/5001241 [==============================] - 29s 6us/step - loss: 62.2818 - val_loss: 24.4431
Epoch 9/10
5001241/5001241 [==============================] 

In [25]:
model.save('mlp1-call40.h5')

In [16]:
call_y_pred = model.predict(call_X_test)

In [22]:
diff = (call_y_test.values - call_y_pred.reshape(call_y_pred.shape[0]))

In [24]:
np.mean(np.square(diff))

27.774671477538476